In [1]:
import pandas as pd
import pm_lib
import ipywidgets as widgets
from IPython.display import clear_output, display
from tkinter import Tk, filedialog

In [2]:
files = ''

def select_files(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    files = filedialog.askopenfilename(title='Choose a File') 

def select_files_multiple(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    files = filedialog.askopenfilename(title='Choose a File', multiple=True) 

fileselect = widgets.Button(description="File select")
fileselect.on_click(select_files)

fileselect_multiple = widgets.Button(description="File select")
fileselect_multiple.on_click(select_files_multiple)


In [3]:
col_names = ['Word', 'Expected']


def on_button_clicked_nfa(b):
    global files
    results = []
    
    with out1:
        nfa = pm_lib.jffToNFA(files)
        doc = 'nfa.cases'
        g = pm_lib.nfa_to_graph(nfa)
        data = pd.read_csv(doc, sep='\t', dtype=object, names = col_names)
        results = pm_lib.nfa_validate(nfa, doc)
        df = pd.DataFrame(columns=["Received"])
        df["Received"] = results
        df = pd.concat([data, df], axis=1)
        df = df.dropna()
        df.to_csv('file_sent.csv', sep='\t', encoding='utf-8', index=False)
        clear_output()
        display(g)
        display(df)

def on_button_clicked_dfa(b):
    global files
    results = []
    
    with out2:
        dfa = pm_lib.jffToDFA(files)
        doc = 'dfa.cases'
        g = pm_lib.dfa_to_graph(dfa)
        data = pd.read_csv(doc, sep='\t', dtype=object, names = col_names)
        results = pm_lib.dfa_validate(dfa, doc)
        df = pd.DataFrame(columns=["Received"])
        df["Received"] = results
        df = pd.concat([data, df], axis=1)
        df = df.dropna()
        df.to_csv('file_sent.csv', sep='\t', encoding='utf-8', index=False)
        clear_output()
        display(g)
        display(df)



In [6]:
def on_button_clicked_union(b):
    global files

    with out3:
        automata1 = pm_lib.jffToDFA(files[0])
        automata2 = pm_lib.jffToDFA(files[1])
        united_automata = pm_lib.automata_dfa_union(automata1, automata2)
        g = pm_lib.dfa_to_graph(united_automata)
        clear_output()
        display(g)

def on_button_clicked_intersection(b):
    global files

    with out4:
        automata1 = pm_lib.jffToNFA(files[0])
        automata2 = pm_lib.jffToNFA(files[1])
        intersection_automata = pm_lib.automata_intersection(automata1, automata2)
        g = pm_lib.nfa_to_graph(intersection_automata)
        clear_output()
        display(g)


In [7]:
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()

button_nfa = widgets.Button(description='Eval')
button_dfa = widgets.Button(description='Eval')
button_union = widgets.Button(description='Union')
button_intersection = widgets.Button(description='Intersection')

button_nfa.on_click(on_button_clicked_nfa)
button_dfa.on_click(on_button_clicked_dfa)
button_union.on_click(on_button_clicked_union)
button_intersection.on_click(on_button_clicked_intersection)
    
vbox1 = widgets.VBox([fileselect, out1, button_nfa])
vbox2 = widgets.VBox([fileselect, out2, button_dfa])
vbox3 = widgets.VBox([fileselect_multiple, out3, button_union])
vbox4 = widgets.VBox([fileselect_multiple, out4, button_intersection])

tab = widgets.Tab(children = [vbox1, vbox2, vbox3, vbox4])
tab.set_title(0, 'NFA')
tab.set_title(1, 'DFA')
tab.set_title(2, 'Union')
tab.set_title(3, 'Intersection')

display(tab)

